# UXsim demo (for Google Colab)

This notebook demonstrates basic functionaries of [UXsim](https://github.com/toruseo/UXsim). 
For further details, please see the [GitHub repo](https://github.com/toruseo/UXsim) and the [technical documentation](https://toruseo.jp/UXsim/docs/index.html).

We show two examples: simple one and large-scale one.

## Simple Example

Here, we show a simple traffic simulation in a Y-shaped network.

First, install UXsim using pip and import the required modules.

In [ ]:
!pip install uxsim

In [ ]:
from uxsim import *
import pandas as pd

### Scenario Definition

First, we will define the main simulation `W`.
The unit of time is s (seconds) and the unit of length is m.

In [ ]:
W = World(
    name="simple_demo",    # Scenario name. Can be blank. Used as the folder name for saving results.
    deltan=5,   # Simulation aggregation unit Δn. Defines how many vehicles are grouped together (i.e., platoon size) for computation. Computation cost is generally inversely proportional to deltan^2.
    tmax=1200,  # Total simulation time (s)
    print_mode=1, save_mode=1, show_mode=1,    # Various options. print_mode determines whether to print information. Usually set to 1, but recommended 0 when running multiple simulations automatically. save_mode determines if visualization results are saved. show_mode determines if visualization results are displayed. It's good to set show_mode=1 on Jupyter Notebook, otherwise recommended 0.
    random_seed=0    # Set the random seed. Specify if you want repeatable experiments. If not, set to None. On Jupyter Notebook, randomness might not always be consistent (requires a fix).
)

The simulation scenario (network structure and demand) is defined.
First, nodes are added, then links are defined to connect the nodes, and finally the traffic demand between the nodes is specified.
In this case, we define a simple Y-shaped merging network.

In [ ]:
W.addNode("orig1", 0, 0) #Create a node. Parameters: node name, visualization x-coordinate, visualization y-coordinate
W.addNode("orig2", 0, 2)
W.addNode("merge", 1, 1)
W.addNode("dest", 2, 1)

W.addLink("link1", "orig1", "merge", length=1000, free_flow_speed=20, jam_density=0.2, merge_priority=0.5) # Create a link. Parameters: link name, start node, end node, length, free_flow_speed, jam_density, merge_priority during merging
W.addLink("link2", "orig2", "merge", length=1000, free_flow_speed=20, jam_density=0.2, merge_priority=2)
W.addLink("link3", "merge", "dest", length=1000, free_flow_speed=20, jam_density=0.2)

W.adddemand("orig1", "dest", 0, 1000, 0.4) # Create OD traffic demand. Parameters: origin node, destination node, start time, end time, demand flow rate
W.adddemand("orig2", "dest", 500, 1000, 0.6)

The network shape can be confirmed as follows.

In [ ]:
W.show_network()

### Simulation Execution

After defining the scenario, you can execute the simulation with `W.exec_simulation()`. This time, the simulation is run to the end. The simulation time, the number of vehicles in the network at that time, their average speed, and the computation time are displayed.

In [ ]:
W.exec_simulation()

### Results

The `W.analyzer` class is responsible for analyzing the results.

A summary of the results can be printed below. Delay ratio is the ratio of delay time to total trip time, with a value close to zero indicating smooth traffic (when the shortest route can be traveled without congestion) and a larger value indicating congestion (when the shortest route is bypassed or congested).

In [ ]:
W.analyzer.print_simple_stats()

### Visualization of Results

#### Network-level

In order to see overall results, you can generate a gif animation of the traffic situation of the entire network.
By default, you can visualize the traffic situation per link, the traffic situation per section within a link (may not be very clear depending on the network geometry).
The thicker the width of the link, the greater the number and density of vehicles, and the darker the color, the lower the speed.
Note that the animation generation speed for large scenarios can be very slow.

In [ ]:
W.analyzer.network_anim(animation_speed_inverse=15, timestep_skip=30, detailed=0, network_font_size=0, figsize=(4,4))
W.analyzer.network_anim(detailed=1, network_font_size=0, figsize=(4,4))

from IPython.display import display, Image
with open("outsimple_demo/anim_network0.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))
with open("outsimple_demo/anim_network1.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))

You can also see the trajectories of some vehicles traveling through the network. This is quite slow.

In [ ]:
W.analyzer.network_fancy(animation_speed_inverse=15, sample_ratio=0.3, interval=3, trace_length=5)
with open("outsimple_demo/anim_network_fancy.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))


If you don't need animation, you can quickly visualize snapshots of network traffic.

In [ ]:
for t in list(range(0,W.TMAX,int(W.TMAX/6))):
    W.analyzer.network(t, detailed=0, network_font_size=0, figsize=(4,4))
for t in list(range(0,W.TMAX,int(W.TMAX/6))):
    W.analyzer.network(t, detailed=1, network_font_size=0, figsize=(4,4))

#### Link-level
In order to investigate detailed traffic dynamics, time-space diagrams (which is a standard visualization method in traffic flow theory) of density and trajectories for each link can be plotted as follows.
Ones for consecutive links can also be plotted.

In [ ]:
W.analyzer.time_space_diagram_density()
W.analyzer.time_space_diagram_traj()
W.analyzer.time_space_diagram_traj_links([["link1", "link3"], ["link2", "link3"]])

#### Area-level

The macroscopic fundamental diagram can also be generated

In [ ]:
W.analyzer.macroscopic_fundamental_diagram()

### Numerical analysis

If you need to compute numbers, you can output the simulation results as pandas.Dataframe format.

In [ ]:
#overall
df = W.analyzer.basic_to_pandas()
display(df)

#OD-specific traffic situation
df = W.analyzer.od_to_pandas()
display(df)

#MFD
df = W.analyzer.mfd_to_pandas()
display(df)

#link-level
df = W.analyzer.link_to_pandas()
display(df)

#within link
df = W.analyzer.link_traffic_state_to_pandas()
display(df)

#vehicle-level
df = W.analyzer.vehicles_to_pandas()
display(df)

Or save as CSV files.

In [ ]:
W.analyzer.output_data()

## Large-scale Example

Noe let's see large-scale scenario with a lot of vehicles in a large network.

If you haven't, please install UXsim first.

In [ ]:
!pip install uxsim

In [ ]:
from uxsim import *
import pandas as pd

At this time, we automatically generate a grid-shaped network.

In [ ]:
W = World(
    name="large",
    deltan=5,
    tmax=7200,
    print_mode=1, save_mode=1, show_mode=0,
    random_seed=0,
)

# create nodes on an imax x jmax grid.
imax = 11
jmax = 11
nodes = {}
for i in range(imax):
    for j in range(jmax):
        nodes[i,j] = W.addNode(f"n{(i,j)}", i, j)

# create links between neighborhood nodes
links = {}
for i in range(imax):
    for j in range(jmax):
        if i != imax-1:
            links[i,j,i+1,j] = W.addLink(f"l{(i,j,i+1,j)}", nodes[i,j], nodes[i+1,j], length=1000, free_flow_speed=20, jam_density=0.2)
        if i != 0:
            links[i,j,i-1,j] = W.addLink(f"l{(i,j,i-1,j)}", nodes[i,j], nodes[i-1,j], length=1000, free_flow_speed=20, jam_density=0.2)
        if j != jmax-1:
            links[i,j,i,j+1] = W.addLink(f"l{(i,j,i,j+1)}", nodes[i,j], nodes[i,j+1], length=1000, free_flow_speed=20, jam_density=0.2)
        if j != 0:
            links[i,j,i,j-1] = W.addLink(f"l{(i,j,i,j-1)}", nodes[i,j], nodes[i,j-1], length=1000, free_flow_speed=20, jam_density=0.2)

# generate demand from each boundary node to the other boundary nodes
demand_flow = 0.035
demand_duration = 3600
for n1 in [(0,j) for j in range(jmax)]:
    for n2 in [(imax-1,j) for j in range(jmax)]:
        W.adddemand(nodes[n2], nodes[n1], 0, demand_duration, demand_flow)
        W.adddemand(nodes[n1], nodes[n2], 0, demand_duration, demand_flow)
for n1 in [(i,0) for i in range(imax)]:
    for n2 in [(i,jmax-1) for i in range(imax)]:
        W.adddemand(nodes[n2], nodes[n1], 0, demand_duration, demand_flow)
        W.adddemand(nodes[n1], nodes[n2], 0, demand_duration, demand_flow)

W.show_network(network_font_size=0)

Note that each link is 1 km long, so this is a 10 km x 10 km area. And the total number of vehicles is about 60000.

In [ ]:
W.exec_simulation()

Let's visualize the network traffic states. You can see complicated traffic pattern emerges in the network.

In [ ]:
W.analyzer.network_anim(animation_speed_inverse=15, timestep_skip=30, detailed=0, network_font_size=0, figsize=(6,6))

from IPython.display import display, Image
with open("outlarge/anim_network0.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))

In [ ]:
W.analyzer.network_fancy(animation_speed_inverse=15, sample_ratio=0.3, interval=3, trace_length=5)

with open("outlarge/anim_network_fancy.gif", "rb") as f:
    display(Image(data=f.read(), format='png'))